# Import

In [1]:
import json
import trimesh
import numpy as np
np.random.seed(42)
import quaternion
import os
import open3d as o3d

import pandas as pd 
import plotly.graph_objs as go
import plotly.io as pio

cat2color_map = {
    'sofa': 'rgba(244, 22, 100, 1)',    # 粉红色
    'table': 'rgba(22, 244, 100, 1)',   # 绿色
    'chair': 'rgba(100, 22, 244, 1)',   # 紫色
    'bed': 'rgba(244, 100, 22, 1)',      # 橙色
    'bookshelf': 'rgba(22, 100, 244, 1)',  # 蓝色
    'cabinet': 'rgba(100, 244, 22, 1)'     # 青色
}

pcd_color = "rgba(10, 10, 10, 0.5)"

def get_info_from_latent_name(l_name):
    # input should be like: 1a4a8592046253ab5ff61a3a2a0e2484_scene0484_00_ins_1.pth
    l_name = os.path.basename(l_name)
    latent_name = l_name.split(".")[0]
    scene = latent_name.split("_")[1]+"_"+latent_name.split("_")[2]
    ins_id = latent_name.split("_")[-1]
    obj_id = latent_name.split("_")[0]
    return scene, ins_id, obj_id

def load_pcd_vis(pcd_path,pcd_name=None,sub_sample=50000) -> np.ndarray:
    _, file_extension = os.path.splitext(pcd_path)
    if file_extension == ".pcd":
        pcd = o3d.io.read_point_cloud(pcd_path)
        point_clouds_data = np.asarray(pcd.points)
    elif file_extension == ".txt":
        point_clouds_data = np.loadtxt(pcd_path)
    elif file_extension == ".npy":
        point_clouds_data = np.load(pcd_path)
    else:
        raise NotImplementedError
    
    N = point_clouds_data.shape[0]
    
    point_clouds_data = point_clouds_data[np.random.choice(N, sub_sample, replace=True), :]
    
    x = point_clouds_data[:, 0]
    y = point_clouds_data[:, 1]
    z = point_clouds_data[:, 2]
    trace = go.Scatter3d(x=x, y=y, z=z, 
            mode='markers',     
            marker=dict(
                size=0.8,  # Adjust the size of the markers here
                color=pcd_color  # Set the color you want (e.g., light blue)
            ),
            name=pcd_name)
    return trace

def plotly_mesh3d_to_trimesh(plotly_mesh3d):
    """
    将Plotly Mesh3d对象转换为trimesh.Trimesh对象。
    
    参数:
        plotly_mesh3d (plotly.graph_objs.Mesh3d): Plotly Mesh3d对象。
        
    返回:
        trimesh.Trimesh: 转换后的trimesh对象。
    """
    # 从Plotly Mesh3d对象提取顶点坐标
    vertices = np.column_stack((plotly_mesh3d.x, plotly_mesh3d.y, plotly_mesh3d.z))
    
    # 从Plotly Mesh3d对象提取面
    faces = np.column_stack((plotly_mesh3d.i, plotly_mesh3d.j, plotly_mesh3d.k))
    
    # 创建trimesh.Trimesh对象
    trimesh_obj = trimesh.Trimesh(vertices=vertices, faces=faces)
    
    return trimesh_obj
    
def BoundingCubeNormalization(mesh,buffer=1.03):
    """
    transferred from deepsdf preprocessing
    input: mesh: trimesh.load
    """
    vertices = np.array(mesh.vertices)
    faces = np.array(mesh.faces)

    xmax = vertices[:,0].max()
    ymax = vertices[:,1].max()
    zmax = vertices[:,2].max()

    xmin = vertices[:,0].min()
    ymin = vertices[:,1].min()
    zmin = vertices[:,2].min()

    xcenter = (xmax+xmin)/2.0
    ycenter = (ymax+ymin)/2.0
    zcenter = (zmax+zmin)/2.0

    vertices[:,0] -= xcenter
    vertices[:,1] -= ycenter 
    vertices[:,2] -= zcenter

    norms = np.linalg.norm(vertices, axis=1)

    max_dist = norms.max()
    max_dist *= buffer

    vertices /= max_dist

    x, y, z = vertices.T  # Transposed for easier unpacking
    i, j, k = faces.T  # Unpack faces

    # if mesh_color is None:
    mesh_color = "rgba(244,22,100,0.5)"
    mesh_name = "normalized"

    mesh_normalized = go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        opacity=0.5,
        color=mesh_color,
        name=mesh_name
    )
    
    return mesh_normalized, 1./max_dist, np.array([xcenter,ycenter,zcenter])

def BoundingCubeDeNormalization(mesh, max_dists_d, centers, buffer=1.03):
    """
    transferred from deepsdf preprocessing
    input: mesh: trimesh.load
    """
    vertices = np.array(mesh.vertices)
    faces = np.array(mesh.faces)

    max_dist = 1./max_dists_d

    max_dist /= buffer

    vertices *= max_dist

    vertices[:,0] += centers[0]
    vertices[:,1] += centers[1]
    vertices[:,2] += centers[2]

    x, y, z = vertices.T  # Transposed for easier unpacking
    i, j, k = faces.T  # Unpack faces

    # if mesh_color is None:
    mesh_color = "rgba(244,22,100,0.5)"
    mesh_name = "unnormalized"

    mesh_unnormalized = go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        opacity=0.5,
        color=mesh_color,
        name=mesh_name
    )
    
    return mesh_unnormalized

def quaternion_list2rotmat(quant_list: list, format="xyzw"):
    assert len(quant_list) == 4, "Quaternion needs 4 elements"
    if format=="xyzw":
        q = np.quaternion(quant_list[0], quant_list[1], quant_list[2], quant_list[3])
    elif format=="wxyz":
        q = np.quaternion(quant_list[1], quant_list[2], quant_list[3], quant_list[0])
    R = quaternion.as_rotation_matrix(q)
    return R

def mesh_apply_rts(mesh, rotation_mat_c2w=None, translation_c2w=None, scale_c2w=None, mesh_name=None, mesh_color=None):
    vertices = np.array(mesh.vertices)
    faces = np.array(mesh.faces)
    
    
    if rotation_mat_c2w is None:
    # Apply rotation
    # pcd_meshcoord = (np.linalg.inv(gt_rotation_mat_c2w) @ (pcd_world - gt_translation_c2w[np.newaxis, :]).T).T
        transformed_vertices = vertices
    else:
        transformed_vertices = (rotation_mat_c2w @ vertices.T).T + translation_c2w[np.newaxis, :]
    
    x, y, z = transformed_vertices.T  # Transposed for easier unpacking
    i, j, k = faces.T  # Unpack faces

    if mesh_color is None:
        mesh_color = "rgba(244,22,100,0.5)"

    mesh_transformed = go.Mesh3d(
        x=x, y=y, z=z,
        i=i, j=j, k=k,
        opacity=0.5,
        color=mesh_color,
        name=mesh_name
    )
    return mesh_transformed

def mesh_load(mesh_path, scale_c2w=None, rotation_quat_wxyz=None, translation_c2w=None,mesh_name=None):
    # 从文件加载网格数据
    file_suffix = mesh_path.split(".")[-1]
    if file_suffix == "obj":
        with open(mesh_path, 'r') as file:
            lines = file.readlines()

        vertices = []
        faces = []

        for line in lines:
            if line.startswith('v '):
                vertex = line.split()[1:]
                vertices.append([float(vertex[0]), float(vertex[1]), float(vertex[2])])
            elif line.startswith('f '):
                face = line.split()[1:]
                face_indices = [int(idx.split('/')[0]) - 1 for idx in face]
                faces.append(face_indices)

        mesh = go.Mesh3d(x=[v[0] for v in vertices], y=[v[1] for v in vertices], z=[v[2] for v in vertices],
                        i=[f[0] for f in faces], j=[f[1] for f in faces], k=[f[2] for f in faces], name=mesh_name)
        return mesh

    elif file_suffix == "ply":
        from plyfile import PlyData

        # 从PLY文件加载网格数据
        plydata = PlyData.read(mesh_path)

        # 提取顶点坐标
        vertices = np.array([list(vertex) for vertex in plydata['vertex'].data])

        # 提取面数据
        faces = np.array(plydata['face'].data['vertex_indices'])
        faces = np.array([list(row) for row in faces])

        # 创建网格图形对象
        mesh = go.Mesh3d(x=vertices[:, 0], y=vertices[:, 1], z=vertices[:, 2], i=faces[:, 0], j=faces[:, 1], k=faces[:, 2], name=mesh_name)
        return mesh

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
lat_name = ""

pcd_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/pcd"

mesh_noneighbor_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"

mesh_neighbor_root_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/69999/test/mesh/Meshes"

In [ ]:
mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/69999/test/mesh/Meshes/2abe8d5b5d29ff52f7628281ecb18112_scene0165_01_ins_3.ply.ply"
mesh_neighbor = mesh_load(mesh_neighbor_path)

basename = os.path.basename(mesh_neighbor_path).split(".")[0]

pcd_path = os.path.join(pcd_root,"{}.pth.pcd".format(basename))

mesh_noneighbor_path = os.path.join(mesh_noneighbor_root,"{}.ply.ply".format(basename))
mesh_noneighbor = mesh_load(mesh_noneighbor_path)

pcd = o3d.io.read_point_cloud(pcd_path)
points = np.asarray(pcd.points)
trace = go.Scatter3d(
    x=points[:, 0],  # X 坐标
    y=points[:, 1],  # Y 坐标
    z=points[:, 2],  # Z 坐标
    mode='markers',
    marker=dict(
        size=2,
        opacity=0.8
    )
)

vis_data = [mesh_neighbor,mesh_noneighbor, trace]

layout = go.Layout(scene=dict(
        aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
        xaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        yaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        zaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
    ),
    showlegend=False
    )

fig = go.Figure(data=vis_data, layout=layout)

fig.show()


# Sofa

In [ ]:
pcd_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/pcd"
mesh_noneighbor_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"

mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/69999/test/mesh/Meshes/2abe8d5b5d29ff52f7628281ecb18112_scene0165_01_ins_2.ply.ply"
mesh_neighbor = mesh_load(mesh_neighbor_path)

basename = os.path.basename(mesh_neighbor_path).split(".")[0]

pcd_path = os.path.join(pcd_root,"{}.pth.pcd".format(basename))

mesh_noneighbor_path = os.path.join(mesh_noneighbor_root,"{}.ply.ply".format(basename))
mesh_noneighbor = mesh_load(mesh_noneighbor_path)

pcd = o3d.io.read_point_cloud(pcd_path)
points = np.asarray(pcd.points)
trace = go.Scatter3d(
    x=points[:, 0],  # X 坐标
    y=points[:, 1],  # Y 坐标
    z=points[:, 2],  # Z 坐标
    mode='markers',
    marker=dict(
        size=2,
        opacity=0.8,
        color=pcd_color
    )
)

layout = go.Layout(scene=dict(
        aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
        xaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        yaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        zaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
    ),
    width=600,
    height=600,
    showlegend=False
    )

vis_data = [mesh_neighbor,mesh_noneighbor, trace]
fig = go.Figure(data=vis_data, layout=layout)
fig.show()


In [5]:
print(mesh_neighbor_path)
print(mesh_noneighbor_path)
print(pcd_path)

/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/69999/test/mesh/Meshes/2abe8d5b5d29ff52f7628281ecb18112_scene0165_01_ins_2.ply.ply
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes/2abe8d5b5d29ff52f7628281ecb18112_scene0165_01_ins_2.ply.ply
/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/pcd/2abe8d5b5d29ff52f7628281ecb18112_scene0165_01_ins_2.pth.pcd


In [ ]:
vis_data = [mesh_noneighbor, trace]

mesh_noneighbor['color'] = cat2color_map["sofa"]
fig = go.Figure(data=vis_data, layout=layout)
fig.show()

In [ ]:
vis_data = [mesh_neighbor, trace]

mesh_neighbor['color'] = cat2color_map["chair"]
fig = go.Figure(data=vis_data, layout=layout)
fig.show()

In [ ]:
pcd_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/pcd"
mesh_noneighbor_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"

mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/69999/test/mesh/Meshes/a930d381392ff51140b559ef47048b86_scene0181_00_ins_8.ply.ply"
mesh_neighbor = mesh_load(mesh_neighbor_path)

basename = os.path.basename(mesh_neighbor_path).split(".")[0]

pcd_path = os.path.join(pcd_root,"{}.pth.pcd".format(basename))

mesh_noneighbor_path = os.path.join(mesh_noneighbor_root,"{}.ply.ply".format(basename))
mesh_noneighbor = mesh_load(mesh_noneighbor_path)

pcd = o3d.io.read_point_cloud(pcd_path)
points = np.asarray(pcd.points)
trace = go.Scatter3d(
    x=points[:, 0],  # X 坐标
    y=points[:, 1],  # Y 坐标
    z=points[:, 2],  # Z 坐标
    mode='markers',
    marker=dict(
        size=2,
        opacity=0.8,
        color=pcd_color
    )
)

layout = go.Layout(scene=dict(
        aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
        xaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        yaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        zaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
    ),
    width=600,
    height=600,
    showlegend=False
    )

vis_data = [mesh_neighbor,mesh_noneighbor, trace]
fig = go.Figure(data=vis_data, layout=layout)
fig.show()


In [ ]:
vis_data = [mesh_noneighbor, trace]
mesh_noneighbor['color'] = cat2color_map['sofa']
fig = go.Figure(data=vis_data, layout=layout)
fig.show()

In [ ]:
mesh_neighbor_path

# Table

In [25]:
# mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/69999/test/mesh/Meshes/7ae657b39aa2be68ccd1bcd57588acf8_scene0517_01_ins_6.ply.ply"

mesh_noneighbor_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_table_train_noneighbor/output/50999/test/mesh/Meshes"
pcd_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_table_train_noneighbor/pcd"

mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_table_train_neighbor/output/23999/test/mesh/2fced9721464b4d9c955e5ed03ef3a2f_scene0221_01_ins_3.ply.ply"

# mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/output/69999/test/mesh/Meshes/2abe8d5b5d29ff52f7628281ecb18112_scene0165_01_ins_2.ply.ply"
mesh_neighbor = mesh_load(mesh_neighbor_path)

basename = os.path.basename(mesh_neighbor_path).split(".")[0]

pcd_path = os.path.join(pcd_root,"{}.pth.pcd".format(basename))

mesh_noneighbor_path = os.path.join(mesh_noneighbor_root,"{}.ply.ply".format(basename))
mesh_noneighbor = mesh_load(mesh_noneighbor_path)

pcd = o3d.io.read_point_cloud(pcd_path)
points = np.asarray(pcd.points)
trace = go.Scatter3d(
    x=points[:, 0],  # X 坐标
    y=points[:, 1],  # Y 坐标
    z=points[:, 2],  # Z 坐标
    mode='markers',
    marker=dict(
        size=2,
        opacity=0.8,
        color=pcd_color
    )
)

layout = go.Layout(scene=dict(
        aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
        xaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        yaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        zaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
    ),
    width=600,
    height=600,
    showlegend=False
    )


In [ ]:
vis_data = [mesh_neighbor, trace]
fig = go.Figure(data=vis_data, layout=layout)
fig.show()

# Chair

In [6]:
mesh_noneighbor_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_noneighbor/output/18999/test/mesh/oldMeshes"
pcd_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_noneighbor/pcd"

mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_neighbor/output/18999/test/mesh/Meshes/1bcd9c3fe6c9087e593ebeeedbff73b_scene0331_01_ins_9.ply.ply"
mesh_neighbor = mesh_load(mesh_neighbor_path)

basename = os.path.basename(mesh_neighbor_path).split(".")[0]

pcd_path = os.path.join(pcd_root,"{}.pth.pcd".format(basename))

mesh_noneighbor_path = os.path.join(mesh_noneighbor_root,"{}.ply.ply".format(basename))
mesh_noneighbor = mesh_load(mesh_noneighbor_path)

pcd = o3d.io.read_point_cloud(pcd_path)
points = np.asarray(pcd.points)
trace = go.Scatter3d(
    x=points[:, 0],  # X 坐标
    y=points[:, 1],  # Y 坐标
    z=points[:, 2],  # Z 坐标
    mode='markers',
    marker=dict(
        size=2,
        opacity=0.8,
        color=pcd_color
    )
)

layout = go.Layout(scene=dict(
        aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
        xaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        yaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        zaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
    ),
    width=600,
    height=600,
    showlegend=False
    )


In [ ]:
vis_data = [mesh_neighbor, trace]
fig = go.Figure(data=vis_data, layout=layout)
fig.show()
pio.write_html(fig,"/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/metrics/chair_neighbor.html")

In [ ]:
vis_data = [mesh_noneighbor, trace]
mesh_noneighbor['color'] = cat2color_map['sofa']
fig = go.Figure(data=vis_data, layout=layout)
fig.show()
pio.write_html(fig,"/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/metrics/chair_noneighbor.html")

# Bed

In [14]:
mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_bed_train_neighbor/output/40999/test/mesh/20b7fd7affe7ef07c370aa5e215a8f19_scene0677_00_ins_1.ply.ply"

mesh_noneighbor_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_bed_train_noneighbor/output/55999/test/mesh"
pcd_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_bed_train_noneighbor/pcd"

mesh_neighbor = mesh_load(mesh_neighbor_path)

basename = os.path.basename(mesh_neighbor_path).split(".")[0]

pcd_path = os.path.join(pcd_root,"{}.pth.pcd".format(basename))

mesh_noneighbor_path = os.path.join(mesh_noneighbor_root,"{}.ply.ply".format(basename))
mesh_noneighbor = mesh_load(mesh_noneighbor_path)

pcd = o3d.io.read_point_cloud(pcd_path)
points = np.asarray(pcd.points)
trace = go.Scatter3d(
    x=points[:, 0],  # X 坐标
    y=points[:, 1],  # Y 坐标
    z=points[:, 2],  # Z 坐标
    mode='markers',
    marker=dict(
        size=2,
        opacity=0.8,
        color=pcd_color
    )
)

layout = go.Layout(scene=dict(
        aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
        xaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        yaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        zaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
    ),
    width=600,
    height=600,
    showlegend=False
    )


In [ ]:
vis_data = [mesh_neighbor, trace]
fig = go.Figure(data=vis_data, layout=layout)
fig.show()
pio.write_html(fig,"/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/metrics/bed_neighbor.html")

In [ ]:
vis_data = [mesh_noneighbor, trace]
fig = go.Figure(data=vis_data, layout=layout)
mesh_noneighbor['color'] = cat2color_map['sofa']
fig.show()
# pio.write_html(fig,"/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/metrics/bed_noneighbor.html")

In [19]:
pio.write_html(fig,"/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/metrics/bed_noneighbor.html")

# Stage 3

## sofa

In [ ]:
pcd_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/pcd"
mesh_noneighbor_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"

mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage3_diff_cond_sofa_train_noneighbor/output/5399/test/mesh/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3.ply.ply"
mesh_neighbor = mesh_load(mesh_neighbor_path)

basename = os.path.basename(mesh_neighbor_path).split(".")[0]

pcd_path = os.path.join(pcd_root,"{}.pth.pcd".format(basename))

mesh_noneighbor_path = os.path.join(mesh_noneighbor_root,"{}.ply.ply".format(basename))
mesh_noneighbor = mesh_load(mesh_noneighbor_path)

pcd = o3d.io.read_point_cloud(pcd_path)
points = np.asarray(pcd.points)
trace = go.Scatter3d(
    x=points[:, 0],  # X 坐标
    y=points[:, 1],  # Y 坐标
    z=points[:, 2],  # Z 坐标
    mode='markers',
    marker=dict(
        size=2,
        opacity=0.8,
        color=pcd_color
    )
)

layout = go.Layout(scene=dict(
        aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
        xaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        yaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        zaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
    ),
    width=600,
    height=600,
    showlegend=False
    )

In [ ]:
vis_data = [mesh_neighbor, trace]
mesh_neighbor['color'] = cat2color_map['sofa']
fig = go.Figure(data=vis_data, layout=layout)
fig.show()

In [ ]:
vis_data = [mesh_noneighbor, trace]
fig = go.Figure(data=vis_data, layout=layout)
fig.show()

In [24]:
mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage3_diff_cond_sofa_train_noneighbor/output/5399/test/mesh/8a9630332c2cb42644ac83b3e97ad658_scene0344_00_ins_6.ply.ply"

pcd_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/pcd"
mesh_noneighbor_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"

# mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage3_diff_cond_sofa_train_noneighbor/output/5399/test/mesh/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3.ply.ply"
mesh_neighbor = mesh_load(mesh_neighbor_path)

basename = os.path.basename(mesh_neighbor_path).split(".")[0]

pcd_path = os.path.join(pcd_root,"{}.pth.pcd".format(basename))

mesh_noneighbor_path = os.path.join(mesh_noneighbor_root,"{}.ply.ply".format(basename))
mesh_noneighbor = mesh_load(mesh_noneighbor_path)

pcd = o3d.io.read_point_cloud(pcd_path)
points = np.asarray(pcd.points)
trace = go.Scatter3d(
    x=points[:, 0],  # X 坐标
    y=points[:, 1],  # Y 坐标
    z=points[:, 2],  # Z 坐标
    mode='markers',
    marker=dict(
        size=2,
        opacity=0.8,
        color=pcd_color
    )
)

layout = go.Layout(scene=dict(
        aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
        xaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        yaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        zaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
    ),
    width=600,
    height=600,
    showlegend=False
    )

In [ ]:
vis_data = [mesh_neighbor, trace]
mesh_neighbor['color'] = cat2color_map['sofa']
fig = go.Figure(data=vis_data, layout=layout)
fig.show()

In [ ]:
vis_data = [mesh_noneighbor, trace]
# mesh_neighbor['color'] = cat2color_map['sofa']
fig = go.Figure(data=vis_data, layout=layout)
fig.show()

In [9]:
mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage3_diff_cond_sofa_train_noneighbor/output/5399/test/mesh/3746c2c38faaec395304a08899f58b0b_scene0022_01_ins_5.ply.ply"

pcd_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/pcd"
mesh_noneighbor_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes"

# mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage3_diff_cond_sofa_train_noneighbor/output/5399/test/mesh/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3.ply.ply"
mesh_neighbor = mesh_load(mesh_neighbor_path)

basename = os.path.basename(mesh_neighbor_path).split(".")[0]

pcd_path = os.path.join(pcd_root,"{}.pth.pcd".format(basename))

mesh_noneighbor_path = os.path.join(mesh_noneighbor_root,"{}.ply.ply".format(basename))
mesh_noneighbor = mesh_load(mesh_noneighbor_path)

pcd = o3d.io.read_point_cloud(pcd_path)
points = np.asarray(pcd.points)
trace = go.Scatter3d(
    x=points[:, 0],  # X 坐标
    y=points[:, 1],  # Y 坐标
    z=points[:, 2],  # Z 坐标
    mode='markers',
    marker=dict(
        size=2,
        opacity=0.8,
        color=pcd_color
    )
)

layout = go.Layout(scene=dict(
        aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
        xaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        yaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        zaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
    ),
    width=600,
    height=600,
    showlegend=False
    )


In [ ]:
vis_data = [mesh_noneighbor, trace]
fig = go.Figure(data=vis_data, layout=layout)
fig.show()

In [ ]:
vis_data = [mesh_neighbor, trace]
mesh_neighbor['color'] = cat2color_map['sofa']
fig = go.Figure(data=vis_data, layout=layout)
fig.show()

# ins

In [ ]:
mesh_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_noneighbor/output/18999/test/mesh/Meshes/1eb2e372a204a61153baab6c8235f5db_scene0297_01_ins_9.ply.ply"
mesh_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_noneighbor/output/18999/test/mesh/Meshes/b1da0d9aab6d2308608505d960f2a393_scene0059_01_ins_21.ply.ply"
mesh_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_chair_train_noneighbor/output/18999/test/mesh/Meshes/94e5d7fb2ff59ff98a94168388287ad5_scene0188_00_ins_11.ply.ply"
mesh = mesh_load(mesh_path)

layout = go.Layout(scene=dict(
        aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
        xaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        yaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        zaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
    ),
    width=600,
    height=600,
    showlegend=False
    )

vis_data = [mesh]
mesh['color'] = cat2color_map['chair']
fig = go.Figure(data=vis_data, layout=layout)
fig.show()


# patch

In [ ]:
import open3d as o3d
import numpy as np
import plotly.graph_objects as go
from sklearn.cluster import DBSCAN

# 读取点云PCD文件
pcd = o3d.io.read_point_cloud(pcd_path)

# 将点云数据转换为numpy数组
points = np.asarray(pcd.points)

# 使用DBSCAN进行聚类
db = DBSCAN(eps=0.02, min_samples=100).fit(points)
labels = db.labels_

# 获取聚类数量
num_clusters = len(set(labels)) - (1 if -1 in labels else 0)

# 生成不同的颜色
colors = px.colors.qualitative.Alphabet

# 创建 Plotly 图形对象
fig = go.Figure()

# 为每个聚类分配颜色
for label in set(labels):
    if label == -1:
        # 噪声点
        color = 'rgba(0, 0, 0, 0.3)'
    else:
        color = colors[label % len(colors)]
    
    cluster_points = points[labels == label]
    fig.add_trace(go.Scatter3d(
        x=cluster_points[:, 0],
        y=cluster_points[:, 1],
        z=cluster_points[:, 2],
        mode='markers',
        marker=dict(size=2, color=color),
        name=f'Cluster {label}'
    ))

# 设置图形布局
fig.update_layout(
    title='Point Cloud Clustering',
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z'
    )
)

# 显示图形
fig.show()


In [21]:
set(labels)

{-1}

# PCD

## diff-ddit

In [2]:
pcd_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/pcd"
mesh_1000_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor_pcd1000test/output/69999/test/mesh/Meshes"
pcd_1000_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa_pcd1000test/pcd"
mesh_128_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor_pcd128test/output/69999/test/mesh/Meshes"
pcd_128_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa_pcd128test/pcd/999"

# mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage3_diff_cond_sofa_train_noneighbor/output/5399/test/mesh/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3.ply.ply"
mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes/2abe8d5b5d29ff52f7628281ecb18112_scene0165_01_ins_2.ply.ply"
mesh_neighbor = mesh_load(mesh_neighbor_path)

basename = os.path.basename(mesh_neighbor_path).split(".")[0]

pcd_path = os.path.join(pcd_root,"{}.pth.pcd".format(basename))
pcd_1000_path = os.path.join(pcd_1000_root,"{}.pth.pcd".format(basename))
pcd_128_path = os.path.join(pcd_128_root,"{}.pth.pcd".format(basename))

mesh_1000_path = os.path.join(mesh_1000_root,"{}.ply.ply".format(basename))
mesh_1000 = mesh_load(mesh_1000_path)

mesh_128_path = os.path.join(mesh_128_root,"{}.ply.ply".format(basename))
mesh_128 = mesh_load(mesh_128_path)

pcd = o3d.io.read_point_cloud(pcd_path)
pcd_1000 = o3d.io.read_point_cloud(pcd_1000_path)
pcd_128 = o3d.io.read_point_cloud(pcd_128_path)

points = np.asarray(pcd.points)
trace = go.Scatter3d(
    x=points[:, 0],  # X 坐标
    y=points[:, 1],  # Y 坐标
    z=points[:, 2],  # Z 坐标
    mode='markers',
    marker=dict(
        size=2,
        opacity=0.8,
        color=pcd_color
    )
)

points = np.asarray(pcd_1000.points)
trace_1000 = go.Scatter3d(
    x=points[:, 0],  # X 坐标
    y=points[:, 1],  # Y 坐标
    z=points[:, 2],  # Z 坐标
    mode='markers',
    marker=dict(
        size=2,
        opacity=0.8,
        color=pcd_color
    )
)

points = np.asarray(pcd_128.points)
trace_128 = go.Scatter3d(
    x=points[:, 0],  # X 坐标
    y=points[:, 1],  # Y 坐标
    z=points[:, 2],  # Z 坐标
    mode='markers',
    marker=dict(
        size=2,
        opacity=0.8,
        color=pcd_color
    )
)

layout = go.Layout(scene=dict(
        aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
        xaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        yaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        zaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
    ),
    width=600,
    height=600,
    showlegend=False
    )

In [ ]:
vis_data = [mesh_neighbor, trace]
mesh_neighbor['color'] = cat2color_map['sofa']
fig = go.Figure(data=vis_data, layout=layout)
fig.show()

In [ ]:
vis_data = [mesh_1000, trace_1000]
mesh_1000['color'] = cat2color_map['sofa']
fig = go.Figure(data=vis_data, layout=layout)
fig.show()

In [ ]:
vis_data = [mesh_128, trace_128]
mesh_128['color'] = cat2color_map['sofa']
fig = go.Figure(data=vis_data, layout=layout)
fig.show()

## ddit

In [12]:
pcd_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_neighbor/pcd"
# mesh_1000_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor_pcd1000test/output/69999/test/mesh/Meshes"
mesh_1000_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa_pcd1000test/output/999/test/mesh/Meshes"
pcd_1000_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa_pcd1000test/pcd"

# mesh_128_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor_pcd128test/output/69999/test/mesh/Meshes"
mesh_128_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa_pcd128test/output/999/test/mesh"
pcd_128_root = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa_pcd128test/pcd/999"

# mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage3_diff_cond_sofa_train_noneighbor/output/5399/test/mesh/7ae657b39aa2be68ccd1bcd57588acf8_scene0362_01_ins_3.ply.ply"
# mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_diff_cond_sofa_train_noneighbor/output/69999/test/mesh/Meshes/2abe8d5b5d29ff52f7628281ecb18112_scene0165_01_ins_2.ply.ply"
mesh_neighbor_path = "/storage/user/huju/transferred/ws_dditnach/Diffusion-SDF/config/ddit_stage2_sofa/output/999/test/mesh/Meshes/2abe8d5b5d29ff52f7628281ecb18112_scene0165_01_ins_2.ply.ply"
mesh_neighbor = mesh_load(mesh_neighbor_path)

basename = os.path.basename(mesh_neighbor_path).split(".")[0]

pcd_path = os.path.join(pcd_root,"{}.pth.pcd".format(basename))
pcd_1000_path = os.path.join(pcd_1000_root,"{}.pth.pcd".format(basename))
pcd_128_path = os.path.join(pcd_128_root,"{}.pth.pcd".format(basename))

mesh_1000_path = os.path.join(mesh_1000_root,"{}.ply.ply".format(basename))
mesh_1000 = mesh_load(mesh_1000_path)

mesh_128_path = os.path.join(mesh_128_root,"{}.ply.ply".format(basename))
mesh_128 = mesh_load(mesh_128_path)

pcd = o3d.io.read_point_cloud(pcd_path)
pcd_1000 = o3d.io.read_point_cloud(pcd_1000_path)
pcd_128 = o3d.io.read_point_cloud(pcd_128_path)

points = np.asarray(pcd.points)
trace = go.Scatter3d(
    x=points[:, 0],  # X 坐标
    y=points[:, 1],  # Y 坐标
    z=points[:, 2],  # Z 坐标
    mode='markers',
    marker=dict(
        size=2,
        opacity=0.8,
        color=pcd_color
    )
)

points = np.asarray(pcd_1000.points)
trace_1000 = go.Scatter3d(
    x=points[:, 0],  # X 坐标
    y=points[:, 1],  # Y 坐标
    z=points[:, 2],  # Z 坐标
    mode='markers',
    marker=dict(
        size=2,
        opacity=0.8,
        color=pcd_color
    )
)

points = np.asarray(pcd_128.points)
trace_128 = go.Scatter3d(
    x=points[:, 0],  # X 坐标
    y=points[:, 1],  # Y 坐标
    z=points[:, 2],  # Z 坐标
    mode='markers',
    marker=dict(
        size=2,
        opacity=0.8,
        color=pcd_color
    )
)

layout = go.Layout(scene=dict(
        aspectmode='data',  # Set the aspect ratio to 'cube' for equal scales
        xaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        yaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
        zaxis=dict(showbackground=False, showgrid=False, showline=False, showticklabels=False, title=''),
    ),
    width=600,
    height=600,
    showlegend=False
    )

In [ ]:
vis_data = [mesh_neighbor, trace]
mesh_neighbor['color'] = cat2color_map['table']
fig = go.Figure(data=vis_data, layout=layout)
fig.show()

In [ ]:
vis_data = [mesh_1000, trace_1000]
mesh_1000['color'] = cat2color_map['table']
fig = go.Figure(data=vis_data, layout=layout)
fig.show()

In [ ]:
vis_data = [mesh_128, trace_128]
mesh_128['color'] = cat2color_map['table']
fig = go.Figure(data=vis_data, layout=layout)
fig.show()